### Manager/Adapater Example 
This example notebook shows off the MastSyncManager and Adapter implementation and how the two concepts can be used to quickly link multiple viewers together and sync their views with the demo ViewSyncUI

In [11]:
from mast_aladin_lite.app import MastAladin
from mast_aladin_lite.adapters import MastSyncManager, ImvizSyncAdapter, AladinSyncAdapter, ViewerSyncUI
from jdaviz import Imviz
import warnings
from ipywidgets import Layout, Box, widgets

#### Initialize Viewers
First, we need to initalize the viewers that we want to use. For this example, we are initializing an **Imviz** viewer and 2 **MastAladin** viewers

In [20]:
imviz = Imviz()
imviz_viewer = imviz.default_viewer

filenames = [
    'jw02727-o002_t062_nircam_clear-f090w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f150w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f200w_i2d.fits',
    'jw02727-o002_t062_nircam_clear-f277w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f356w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f444w_i2d.fits'
]

with warnings.catch_warnings(), imviz.batch_load():
    warnings.simplefilter('ignore')
    for fn in filenames:
        uri = f"mast:JWST/product/{fn}"
        imviz.load_data(uri, cache=True)

INFO:astroquery:Found cached file jw02727-o002_t062_nircam_clear-f090w_i2d.fits with expected size 626028480.
INFO:astroquery:Found cached file jw02727-o002_t062_nircam_clear-f277w_i2d.fits with expected size 144066240.


INFO: Found cached file jw02727-o002_t062_nircam_clear-f090w_i2d.fits with expected size 626028480. [astroquery.query]
INFO: Found cached file jw02727-o002_t062_nircam_clear-f277w_i2d.fits with expected size 144066240. [astroquery.query]


In [13]:
layout = Layout(width="100%")
aladin_viewer_a = MastAladin(target="Cartwheel Galaxy", layout=layout, fov=.05, height=500)
aladin_viewer_b = MastAladin(target="Cartwheel Galaxy", layout=layout, survey="P/DSS2/red", fov=.05, height=500)

box_layout = Layout(
    display="flex", flex_flow="row", align_items="stretch", border="solid", width="100%"
)
box = Box(children=[aladin_viewer_a,aladin_viewer_b], layout=box_layout)

#### MastSyncManager 
Next, we initialize the MastSyncManager . This will be responsible for maintaing the state of registered viewers as well as orchestrating the syncing of viewers through the UI.

In [14]:
manager = MastSyncManager()

We then register each of the viewers we created earlier with the manager. Each viewer should have a unique identifier and use the appropriate **Adapter** based on the type of viewer that it is.

In [15]:
manager.register_viewer("imviz", ImvizSyncAdapter(imviz_viewer))
manager.register_viewer("aladin-a", AladinSyncAdapter(aladin_viewer_a))
manager.register_viewer("aladin-b", AladinSyncAdapter(aladin_viewer_b))

Finally, we create the **ViewerSyncUI** instance which takes the manager to display as an argument

In [16]:
ui = ViewerSyncUI(manager)

#### Demonstration
Now, when you display the viewers, you can utilize the UI to sync the state of one viewer to all other configured viewers. Try it for yourself by running the code below and using the UI.

In [21]:
imviz.show()
imviz.link_data(align_by='wcs')
imviz.plugins['Orientation'].set_north_up_east_left()
box

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/latest/imviz/index.html', events=['cal…

Box(children=(MastAladin(layout=Layout(width='100%')), MastAladin(layout=Layout(width='100%'), survey='P/DSS2/…

In [18]:
ui.display()